## Journals Search Requests
The ADS maintains a database of summary information about the publications we index -- the Journals database (JournalsDB).  It includes information that the ADS uses for internal content management, but that is also useful for librarians and other content managers to know about how specific titles are classified and what the ADS' holdings are for individual titles and volumes.  This notebook explains how to retrieve information from the JournalsDB using the Journals API.

The base URL for journals queries is `https://api.adsabs.harvard.edu/v1/journals/<endpoint>`. 

where the allowed `<endpoint>` values are:
* `/summary/<bibstem>`: generate a summary of information about the publication having `<bibstem>`; note here and elsewhere in this document that bibstem is case-sensitive (`ApJ` is a valid bibstem, `apj` is not)
* `/journal/<search text>`:generate a list of bibstems whose journal name and abbreviation data contain `<search text>`; `<search text>` must be url-encoded text
* `/refsource/<bibstem>`: generate a list of citation data sources on a per volume basis for a given `<bibstem>`
* `/holdings/<bibstem>/<volume>`: generate a list of available electronic sources for a given `<bibstem>/<volume>`
* `/issn/<search text>`: return the ADS bibstem for the journal having an ISSN identifier that matches `<search text>`
* `/browse/<bibstem>`: generate an appreviated summary for a publication having `<bibstem>`

All queries to the API must include an HTTP header that specifies your API access token, e.g.

`curl -H 'Authorization: Bearer <token>' ...`

In the examples below, `<token>` is your personal API access token as described in the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md).

Notes:
* The information in the JournalsDB is being actively curated and expanded, and records may be added or modified without notice.  For questions about results, please contact adshelp@cfa.harvard.edu.
* All examples use the default HTTP method GET.
* the API examples in this notebook generally assume you're using `curl` statements from the shell command line--you'll see `%%bash` at the top of a cell if its contents should be run in the shell instead of in Python. Guidelines for translating bash `curl` statements into Python can be found in the [converting curl to python notebook](Converting_curl_to_python.ipynb).

### `/summary/<bibstem>`
Generate a summary of information about the publication having `<bibstem>`

*Return data*: output is JSON
* `summary`
  * `abbrev` (list, text): names and abbreviations for the Title found in our metadata holdings
  * `idents` (list, dict): identifiers for the Title (currently consist of ISSN (print and electronic) and Crossref set numbers)
    * `"id_type"` (one of "ISSN_print", "ISSN_electronic", or "Crossref")
    * `"id_value"`
  * `master` (dict): canonical record for Title (Note: some fields are missing/incomplete)
    * `bibstem` (text, case-sensitive): the ADS bibstem
    * `collection` (text): ADS collection that includes Title
    * `defunct` (boolean): `True` if publication of Title has ceased
    * `journal_name` (text): canonical name for Title used by ADS
    * `multilingual` (boolean): `True` if Title publishes papers in two or more languages in a single volume
    * `not_indexed` (boolean): ADS internal field, used for retired or reassigned bibstems (e.g. `ApJL`)
    * `notes` (text): curator and other notes about the Title
    * `primary_language` (text): ISO 639-1 code for Title fulltext language
    * `pubtype` (text): one of "Journal", "Conf. Proc.", "Monograph", "Book", "Software", "Other"
    * `refereed` (text): one of "yes", "no", "na", "partial"
  * `names` (list, dict): native-language and transliterated names for non-English Titles
    * `name_english_translated` (text): Title name translated into English
    * `title_language` (text): ISO 639-1 code for the Title's original language
    * `name_native_language` (text): Unicode rendering of the Title in `title_language`
    * `name_normalized` (text): Title `name_native_language` transliterated into English
  * `pubhist` (list, dict): current and historical information about Title and its publisher(s); when possible, Titles having multiple publishers during their history will have multiple entries, each with the appropriate date and volume ranges
    * `publisher` (text): Publisher name or acronym
    * `title` (dict): Summary publication information
      * `complete` (text): `Y` if ADS index the complete run of this publication
      * `notes` (text): Curator or other notes
      * `successor_masterid` (text): identifier of successor Title, if publication ceased and continued under a different name
      * `vol_end` (text): last or most recent available volume from Publisher in ADS records (currently active Titles will have `-`)
      * `vol_start` (text): first available volume from Publisher in ADS records
      * `year_end` (integer): most recent available year from Publisher in ADS records
      * `year_start` (integer): first available year from Publisher in ADS records


In [1]:
%%bash
token="your-token-here"
# note: add | python -m json.tool to the end of the request to print the response in a more readable format
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/journals/summary/PASJ | python3 -m json.tool

{
    "summary": {
        "master": {
            "bibstem": "ApJ",
            "journal_name": "The Astrophysical Journal",
            "primary_language": "en",
            "multilingual": false,
            "defunct": false,
            "pubtype": "Journal",
            "refereed": "yes",
            "collection": null,
            "notes": null,
            "not_indexed": false
        },
        "idents": [
            {
                "id_type": "ISSN_print",
                "id_value": "0004-637X"
            },
            {
                "id_type": "Crossref",
                "id_value": "4876",
            {
                "id_type": "ISSN_electronic",
                "id_value": "1538-4357"
            }
        ],
        "abbrev": [
            "ASTROPHYSICAL JOURNAL",
            "Ap. J.",
            "Ap. J. Lett.",
            "ApJ Letters",
            "ApJL",
            "Astrophy. J.",
            "Astrophys J",
            "Astrophys J Lett Ed",
            "As

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1104  100  1104    0     0   5132      0 --:--:-- --:--:-- --:--:--  5385


### `/journal/<search text>`
Generate a list of bibstems whose journal name and abbreviation data contain `<search text>`

*Return data*: output is JSON
* `journal` (list, dict): list of matching name-bibstem pairs whose names match `<search text>`
  * `bibstem` (text, case-sensitive): ADS bibstem for Title
  * `name` (text): name of Title matching `<search text>`


In [2]:
%%bash
token="your-token-here"
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/journals/journal/Astronom%20Jour | python3 -m json.tool


{
    "journal": [
        {
            "bibstem": "OAJ",
            "name": "The Open Astronomy Journal"
        },
        {
            "bibstem": "AJ",
            "name": "The Astronomical Journal"
        },
        {
            "bibstem": "JRASC",
            "name": "Journal of the Royal Astronomical Society of Canada"
        },
        {
            "bibstem": "AJS",
            "name": "The Astronomical Journal Supplement"
        },
        {
            "bibstem": "CAPJ",
            "name": "Communicating Astronomy with the Public Journal"
        },
        {
            "bibstem": "JBAA",
            "name": "Journal of the British Astronomical Association"
        },
        {
            "bibstem": "BlgAJ",
            "name": "Bulgarian Astronomical Journal"
        },
        {
            "bibstem": "PAIJ",
            "name": "Physics &amp; Astronomy International Journal"
        },
        {
            "bibstem": "RadA",
            "name": "Radio Astronomy:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1151  100  1151    0     0   3905      0 --:--:-- --:--:-- --:--:--  4052


### `/refsource/<bibstem>`
Generate a list of citation data sources on a per volume basis for a given `<bibstem>`

*Return data*: output is JSON
* `refsource` (dict): list of volume, year, and reference sources in ADS holdings for a given bibstem
  *  `bibstem` (text, case-sensitive): ADS bibstem for a given Title
  *  `volumes` (list, dict): refsource data for individual volumes of Title
    * `volume` (text): Volume number
    * `year` (text): Year the volume was published
    * `refsources` (dict): number of papers with reference data originating from each `<data source>` in the volume
      * `<data source>` (text): Origin of references, can be one or more of: [`AUTHOR`, `ISI`, `CROSSREF`, `OCR`, `PUBLISHER` or `OTHER`]
      * `<number of papers>` (integer): count of records having reference data sourced from `<data source>`; if this number is the same as the number of papers in the volume, we have complete references from that source for the Title/volume; `PUBLISHER` data should be considered the most definitive.


In [3]:
%%bash
token="your-token-here"
# note: add | python -m json.tool to the end of the request to print the response in a more readable format
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/journals/refsource/ARA&A' | python3 -m json.tool

{
    "refsource": {
        "bibstem": "ARA&A",
        "volumes": [
            {
                "volume": "1",
                "year": "1963",
                "refsources": {
                    "OCR": 14,
                    "AUTHOR": 8
                }
            },
            {
                "volume": "2",
                "year": "1964",
                "refsources": {
                    "OCR": 17,
                    "AUTHOR": 12
                }
            },
            {
                "volume": "3",
                "year": "1965",
                "refsources": {
                    "OCR": 13,
                    "AUTHOR": 5,
                    "OTHER": 1
                }
            },
            {
                "rows-omitted": "[...]",
            },
            {
                "volume": "57",
                "year": "2019",
                "refsources": {
                    "CROSSREF": 15,
                    "AUTHOR": 1
                }
            },
 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5119  100  5119    0     0  12465      0 --:--:-- --:--:-- --:--:-- 12861


### `/holdings/<bibstem>/<volume>`
Generate a list of available electronic fulltext sources for a given `<bibstem>/<volume>`

*Return data*: output is JSON
* `bibstem` (text, case-sensitive): ADS bibstem for a given Title
* `volume` (text): volume for a given Title (may be alpha-numeric, e.g. `500A`)
* `numFound` (integer): Number of records found for Title/volume
* `holdings` (list, dict): page number and e-sources for each indexed paper in the volume
  * `page` (text): page number or page id
  * `esources` (list, text): one or more of [`EPRINT_HTML`, `EPRINT_PDF`, `PUB_HTML`, `PUB_PDF`, `ADS_PDF`, `ADS_SCAN`, `AUTHOR_PDF`, `AUTHOR_HTML`]


In [4]:
%%bash
token="your-token-here"
# note: add | python -m json.tool to the end of the request to print the response in a more readable format
curl -H 'Authorization: Bearer $token' 'https://api.adsabs.harvard.edu/v1/journals/holdings/ApJS/257' | python3 -m json.tool

{
    "bibstem": "ApJS",
    "volume": "257",
    "numFound": 71,
    "holdings": [
        {
            "esources": [
                "EPRINT_HTML",
                "EPRINT_PDF",
                "PUB_HTML",
                "PUB_PDF"
            ],
            "page": "1"
        },
        {
            "esources": [
                "EPRINT_HTML",
                "EPRINT_PDF",
                "PUB_HTML",
                "PUB_PDF"
            ],
            "page": "2"
        },
        {
            "esources": [
                "EPRINT_HTML",
                "EPRINT_PDF",
                "PUB_HTML",
                "PUB_PDF"
            ],
            "page": "3"
        },
        {
            "rows-omitted": "[...]"
        },
        {
            "esources": [
                "EPRINT_HTML",
                "EPRINT_PDF",
                "PUB_HTML",
                "PUB_PDF"
            ],
            "page": "69"
        },
        {
            "esources": [
                "P

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5474  100  5474    0     0  14084      0 --:--:-- --:--:-- --:--:-- 14519


### `/issn/<search text>`
Return the ADS Bibstem and Journal Name having the ISSN `<search text>`

*Return data*: output is JSON
* `issn` (dict): dict containing bibstem, journal_name, ISSN, and ISSN_type for the journal with ISSN matching `<search text>`
  * `ISSN` (text, case-sensitive): ISSN for the matching journal
  * `ISSN_type` (text, case-sensitive): Type of ISSN (including `ISSN_print`, `ISSN_electronic`, among other)
  * `bibstem` (text, case-sensitive): bibstem for the matching journal
  * `journal_name` (text): name of the matching journal


In [5]:
%%bash
token="your-token-here"
# note: add | python -m json.tool to the end of the request to print the response in a more readable format
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/journals/issn/0004-637X | python3 -m json.tool

{
    "issn": {
        "ISSN": "0004-637X",
        "ISSN_type": "ISSN_print",
        "bibstem": "ApJ",
        "journal_name": "The Astrophysical Journal"
    }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5474  100  5474    0     0  14084      0 --:--:-- --:--:-- --:--:-- 14519


### `/browse/<search text>`
Generate an abbreviated summary of information about the publication having `<bibstem>`

*Return data*: output is JSON
  * `browse` (dict): dict containing canonical_name, canonical_abbreviation, classic_bibstem, completeness_estimate, external_identifiers, native_language_title, primary_language, publication_history, and title_language for the journal with bibstem matching `<search text>`
    * `canonical_name` (text, case-sensitive): canonical name for the matching journal
    * `classic_bibstem` (text, case-sensitive): ADS Bibliographic Code (bibstem) assigned to the matching journal)
    * `canonical_abbreviation` (text, case-sensitive): canonical abbreviation assigned to the matching journal
    * `primary_language` (text): two-letter code for the language that most or all papers in the matching journal are written in
    * `native_language_title` (text, case-sensitive): Matching journal title rendered in its native language, if different from English
    * `title_language` (text): two-letter code for the language that native_language_title is written in
    * `completeness_estimate` (text): estimated coverage of SciX holdings for the matching journal, where 1.0 means every article in the title is known to be indexed
    * `external_identifiers` (list, dict): known external identifiers for the e matching journal
      * `id_type` (text): Type of external identifier
      * `id_value` (text): External identifier
    * `publication_history` (list, dict): Known publisher(s) and their year and volume coverage, where journal titles published by more than one publishing house may have multiple entries
      * `publisher` (text): Abbreviated name of publisher
      * `start_year` (text): First year of title published by publisher
      * `start_volume` (text): First volume of title published by publisher


In [6]:
%%bash
token="your-token-here"
# note: add | python -m json.tool to the end of the request to print the response in a more readable format
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/journals/browse/ApJ | python3 -m json.tool

{
    "browse": {
        "canonical_name": "The Astrophysical Journal",
        "classic_bibstem": "ApJ",
        "canonical_abbreviation": "",
        "primary_language": "en",
        "native_language_title": "",
        "title_language": "",
        "completeness_estimate": "0.9998",
        "external_identifiers": [
            {
                "id_type": "ISSN_print",
                "id_value": "0004-637X"
            },
            {
                "id_type": "Crossref",
                "id_value": "4876"
            },
            {
                "id_type": "ISSN_electronic",
                "id_value": "1538-4357"
            }
        ],
        "publication_history": [
            {
                "publisher": "UChicago",
                "start_year": 1895,
                "start_volume": "1"
            },
            {
                "publisher": "IOP",
                "start_year": 2009,
                "start_volume": "690"
            }
        ]
    }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   534  100   534    0     0   3793      0 --:--:-- --:--:-- --:--:--  3814
